# Train Vehicles Agents


- crée un network à partir d'un fichier .osm et des trajectoires de véhiculess
- ajoute un flux de voiture sur les routes
- personnalise un Environnement pour le RL
- integre l'environnement pour RLlib et execute la simulation


In [1]:
from flow.core.params import VehicleParams
from flow.core.params import NetParams, SumoCarFollowingParams
from flow.core.params import InitialConfig
from flow.core.params import EnvParams
from flow.core.params import SumoParams
from flow.controllers import RLController, IDMController
from flow.networks import Network
from flow.core.params import InFlows

## Crée le network

Specifie les noms des routes du network dont les vehicules peuvent être s'insérer

In [2]:
EDGES_DISTRIBUTION = ["-100822066","4794817","4783299#0","155558218"]

créer la classe Network pour spécifier les itinéraires possibles

In [3]:
class IssyOSMNetwork(Network):

    def specify_routes(self, net_params):
        return {
            "-100822066": [ #N
                "-100822066",
                "-352962858#1",
                "-352962858#0",
                "-4786940#1",
                 "-4786940#0",
            ],
            
            "4794817" : [ #Loop
                "4794817",
                "4786972#0",
                "4786972#1",
                "4786972#2",
                "4786965#1",
                "4786965#2",
                "4786965#3",
                "4795729",
                "-352962858#1",
                "4795742#0",
                "4795742#1",
                "4786965#3",
                "4786965#4",
                "4786965#5",
            ],
            
            "4783299#0": [    #E
                "4783299#0",
                "4783299#1",
                "4783299#2",
                "4783299#3",
                "4783299#4",
                "4783299#5",
                "4783299#6",
                "4786940#0",
                "4786940#1",
                "352962858#0",
                "352962858#1",
                "100822066",
            ],
            
            "155558218": [
                "155558218",
                "4786940#1",
                "352962858#0",
                "352962858#1",
                "100822066",
            ],     
        }

## Ajoute les flux de voiture

`IDMController` : The Intelligent Driver Model is a car-following model specifying vehicle dynamics by a differential equation for acceleration $\dot{v}$.

`RLController` : a trainable autuonomous vehicle whose actions are dictated by an RL agent. 

In [4]:
vehicles = VehicleParams()
vehicles.add("human",acceleration_controller=(IDMController, {}),num_vehicles=10)
vehicles.add("rl",acceleration_controller=(RLController, {}),num_vehicles=2)

- `vehs_per_hour`: nombre de vehicule par heure, uniformément espacés. Par exemple, comme il y a $60 \times 60 = 3600$ secondes dans une heure, le parametre $\frac{3600}{5}=720$ va faire rentrer des vehicules dans le network toutes les $5$ secondes.

- `probability`: c'est la probabilité qu'un véhicule entre dans le network toutes les secondes. Par exemple, si on la fixe à $0.2$, alors chaque seconde de la simulation un véhicule aura $\frac{1}{5}$ chance d'entrer dans le network

- `period`: C'est le temps en secondes entre 2 véhicules qui sont insérés. Par exemple, le fixer à $5$ ferait rentrer des véhicules dans le network toutes les $5$ secondes (ce qui équivaut à mettre `vehs_per_hour` à $720$).

<font color='red'>
$\rightarrow$ Exactement 1 seul de ces 3 paramètres doit être configurer !
</font>

In [5]:
inflow = InFlows()

inflow.add(veh_type      = "human",
           edge          = "4794817",
           probability   = 0.3, 
           depart_speed  = 7,
           depart_lane   = "random")

inflow.add(veh_type      = "human",
           edge          = "4783299#0",
           probability   = 0.2,
           depart_speed  = 7,
           depart_lane   = "random")

inflow.add(veh_type       = "human",
           edge           = "-100822066",
           probability    = 0.25,
           depart_speed   = 7,
           depart_lane    = "random")

inflow.add(veh_type       = "rl",
           edge           = "-100822066",
           probability    = 0.05,
           depart_speed   = 7,
           depart_lane    = "random",
           color          = "blue")

inflow.add(veh_type       = "human",
           edge          = "155558218",
           probability   = 0.2,
           depart_speed  = 7,
           depart_lane   = "random")

## Personnalise un Environnement pour le RL

plus de méthodes sur : http://berkeleyflow.readthedocs.io/en/latest/

In [6]:
# import the base environment class
from flow.envs import Env
from gym.spaces.box import Box
from gym.spaces import Tuple
import numpy as np

### definition de la classe environnement 

In [7]:
class myEnv(Env):
    pass

### fonction action_space
2 actions possibles pour chaque véhicule RL : +1 acceleration ou -1 acceleration

In [8]:
class myEnv(myEnv): # update my environment class

    @property
    def action_space(self):
        num_actions = self.initial_vehicles.num_rl_vehicles
        accel_ub = self.env_params.additional_params["max_accel"]
        accel_lb = - abs(self.env_params.additional_params["max_decel"])

        return Box(low=accel_lb, high=accel_ub, shape=(num_actions,))

### observation_space
2 valeurs observé pour chaque véhicule: sa **position** et sa **vitesse**. En conséquence, nous avons besoin d'un espace d'observation qui est *deux fois plus grand que le nombre de véhicules* dans le network.

In [9]:
class myEnv(myEnv):  # update my environment class

    @property
    def observation_space(self):
        return Box(
            low=0,
            high=float("inf"),
            shape=(2*self.initial_vehicles.num_vehicles,))

### apply_rl_actions
`apply_rl_actions` : transforme les commandes de l'agent RL en actions réelles du simulateur.  

Pour notre exemple, l'agent RL peut spécifier que les accélérations des véhicules RL avec la fonction **apply_acceleration**

In [10]:
class myEnv(myEnv):  # update my environment class

    def _apply_rl_actions(self, rl_actions):
        # the names of all autonomous (RL) vehicles in the network
        rl_ids = self.k.vehicle.get_rl_ids()

        # use the base environment method to convert actions into accelerations for the rl vehicles
        self.k.vehicle.apply_acceleration(rl_ids, rl_actions)

### get_state

`get_state` : extrait des features de l'environnement et fournit ensuite des entrées à la stratégie fournie par l'agent RL. 

In [11]:
class myEnv(myEnv):  # update my environment class

    def get_state(self, **kwargs):
        # the get_ids() method is used to get the names of all vehicles in the network
        ids = self.k.vehicle.get_ids()

        # we use the get_absolute_position method to get the positions of all vehicles
        pos = [self.k.vehicle.get_x_by_id(veh_id) for veh_id in ids]

        # we use the get_speed method to get the velocities of all vehicles
        vel = [self.k.vehicle.get_speed(veh_id) for veh_id in ids]

        # the speeds and positions are concatenated to produce the state
        return np.concatenate((pos, vel))

### compute_reward

`compute_reward` : renvoie la récompense associée à un état donné. 

Ici, la fonction de récompense est la **vitesse moyenne de tous les véhicules actuellement sur le réseau**.

In [12]:
class myEnv(myEnv):  # update my environment class

    def compute_reward(self, rl_actions, **kwargs):
        # the get_ids() method is used to get the names of all vehicles in the network
        ids = self.k.vehicle.get_ids()

        # we next get a list of the speeds of all vehicles in the network
        speeds = self.k.vehicle.get_speed(ids)

        # finally, we return the average of all these speeds as the reward
        return np.mean(speeds)

## Lance une simulation avec Training RLlib

Pour qu'un environnement puisse être entrainé, l'environnement doit être accessible via l'importation à partir de flow.envs. 


<font color='red'>
Copier l'environnement créé dans un fichier .py et on importe l'environnement dans `flow.envs.__init__.py`.
Mettre le chemin absolu du fichier .osm .
</font> 

In [13]:
import json
import ray
from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env

from flow.networks.ring import ADDITIONAL_NET_PARAMS
from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder
from flow.core.params import VehicleParams, SumoCarFollowingParams

#from flow.envs import AccelEnv as myEnv
#ADDITIONAL_ENV_PARAMS = {"target_velocity": 20,"sort_vehicles": False,"max_accel": 1,"max_decel": 1}
from myenv import myEnv as myEnv
ADDITIONAL_ENV_PARAMS = {"max_accel": 1, "max_decel": 1}

In [14]:
HORIZON = 200

In [15]:
# number of rollouts per training iteration
N_ROLLOUTS = 10
# number of parallel workers
N_CPUS = 2

# SUMO PARAM
sumo_params = SumoParams(sim_step=0.1, render=False, restart_instance=True)

# ENVIRONMENT PARAM
env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS, horizon=HORIZON)

# NETWORK PARAM
additional_net_params = ADDITIONAL_NET_PARAMS.copy()
path_file  = '/home/julien/projet_CIL4SYS/NOTEBOOKS/issy.osm'
net_params = NetParams(inflows=inflow, osm_path=path_file) 

# NETWORK
network = IssyOSMNetwork

# INITIAL CONFIG
initial_config = InitialConfig(edges_distribution=EDGES_DISTRIBUTION)


flow_params = dict( exp_tag   = "ISSY_RL_train", 
                    env_name  = myEnv,  
                    network   = IssyOSMNetwork,
                    simulator = 'traci', # simulator that is used by the experiment
                    sim       = sumo_params,
                    env       = env_params,
                    net       = net_params,
                    veh       = vehicles,
                    initial   = initial_config)

def setup_exps():
    """Return the relevant components of an RLlib experiment.

    Returns
    -------
    str
        name of the training algorithm
    str
        name of the gym environment to be trained
    dict
        training configuration parameters
    """
    alg_run   = "PPO"
    agent_cls = get_agent_class(alg_run)
    config    = agent_cls._default_config.copy()
    config["num_workers"]      = N_CPUS
    config["train_batch_size"] = HORIZON * N_ROLLOUTS
    config["gamma"]            = 0.999  # discount rate
    config["use_gae"]          = True
    config["lambda"]           = 0.97
    config["kl_target"]        = 0.02
    config["num_sgd_iter"]     = 10
    config['clip_actions']     = False  # FIXME(ev) temporary ray bug
    config["horizon"]          = HORIZON
    config["model"].update({"fcnet_hiddens": [3, 3]})
    
    # save the flow params for replay
    flow_json = json.dumps( flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4)
    config['env_config']['flow_params'] = flow_json
    config['env_config']['run'] = alg_run

    create_env, gym_name = make_create_env(params=flow_params, version=0)

    # Register as rllib env
    register_env(gym_name, create_env)
    
    return alg_run, gym_name, config


alg_run, gym_name, config = setup_exps()

ray.init(num_cpus=N_CPUS + 1)

exp_tag = {"run": alg_run,
           "env": gym_name,
           "config": {**config},
           "checkpoint_freq": 500,
           "checkpoint_at_end": True,
           "max_failures": 999,
           "stop": {"training_iteration": 5}}

trials = run_experiments({flow_params["exp_tag"]: exp_tag}, verbose=0)

2020-03-27 14:36:41,040	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2020-03-27_14-36-41_039127_9575/logs.
2020-03-27 14:36:41,171	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:40655 to respond...
2020-03-27 14:36:41,307	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:26486 to respond...
2020-03-27 14:36:41,313	INFO services.py:809 -- Starting Redis shard with 2.09 GB max memory.
2020-03-27 14:36:41,401	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2020-03-27_14-36-41_039127_9575/logs.
2020-03-27 14:36:41,406	INFO services.py:1475 -- Starting the Plasma object store with 3.14 GB memory using /dev/shm.
2020-03-27 14:36:41,731	INFO trial_runner.py:176 -- Starting a new experiment.
2020-03-27 14:36:41,862	WARNING signature.py:108 -- The function with_updates has a **kwargs argument, which is currently not supported.
2020-03-27 14:36:41,921	ERROR log_sync.py:34 -- Log sync require

(pid=9610) 2020-03-27 14:36:50,129	WARNING ppo.py:143 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
(pid=9610) Warning: Discarding unknown compound 'cycleway.opposite' in type 'cycleway.opposite|highway.residential' (first occurence for edge '4786965#0').
(pid=9610) Warning: Discarding unknown compound 'cycleway.opposite_track' in type 'cycleway.opposite_track|highway.residential' (first occurence for edge '4786972#0').
(pid=9610) Warning: Discarding unknown compound 'cycleway.opposite' in type 'cycleway.opposite|highway.living_street' (first occurence for edge '4795758').
(pid=9610) Warning: Discarding unknown compound 'cycleway.lane' in type 'cycleway.lane|highway.secondary' (first occurence for edge '100822066').
(pid=9610) Warning: Removed a road without junctions: -21549451
(pid=9610) Warning: Removed a road without junctions: 21549451
(pid=9610) Warning: The traffic light '30624291' does not control any links; it wi

(pid=9675) 2020-03-27 14:37:08,345	INFO rollout_worker.py:319 -- Creating policy evaluation worker 2 on CPU (please ignore any CUDA init errors)
(pid=9675) 2020-03-27 14:37:08.412835: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2
(pid=9611) /home/julien/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
(pid=9611)   "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
(pid=9610) 2020-03-27 14:37:09,466	INFO trainable.py:105 -- _setup took 20.522 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(pid=9610) 2020-03-27 14:37:09,466	WARNING util.py:47 -- Install gputil for GPU system monitoring.
(pid=9675) /home/julien/anaconda

2020-03-27 14:37:14,118	ERROR trial_runner.py:550 -- Error processing event.
Traceback (most recent call last):
  File "/home/julien/anaconda3/envs/flow/lib/python3.6/site-packages/ray/tune/trial_runner.py", line 498, in _process_trial
    result = self.trial_executor.fetch_result(trial)
  File "/home/julien/anaconda3/envs/flow/lib/python3.6/site-packages/ray/tune/ray_trial_executor.py", line 342, in fetch_result
    result = ray.get(trial_future[0])
  File "/home/julien/anaconda3/envs/flow/lib/python3.6/site-packages/ray/worker.py", line 2247, in get
    raise value
ray.exceptions.RayTaskError: ray_PPO:train() (pid=9610, host=julien-Ubuntu)
  File "/home/julien/anaconda3/envs/flow/lib/python3.6/site-packages/ray/rllib/agents/trainer.py", line 369, in train
    raise e
  File "/home/julien/anaconda3/envs/flow/lib/python3.6/site-packages/ray/rllib/agents/trainer.py", line 358, in train
    result = Trainable.train(self)
  File "/home/julien/anaconda3/envs/flow/lib/python3.6/site-package

(pid=9610) 2020-03-27 14:37:14,111	INFO trainer.py:366 -- Worker crashed during call to train(). To attempt to continue training without the failed worker, set `'ignore_worker_failures': True`.
(pid=9611) 2020-03-27 14:37:14,098	INFO sampler.py:304 -- Raw obs from env: { 0: { 'agent0': np.ndarray((24,), dtype=float64, min=0.0, max=1850.367, mean=336.911)}}
(pid=9611) 2020-03-27 14:37:14,099	INFO sampler.py:305 -- Info return from env: {0: {'agent0': None}}


2020-03-27 14:37:14,396	WARNING util.py:145 -- The `process_trial` operation took 0.28057265281677246 seconds to complete, which may be a performance bottleneck.


(pid=9782) 2020-03-27 14:37:21,711	WARNING ppo.py:143 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
(pid=9782) Warning: Discarding unknown compound 'cycleway.opposite' in type 'cycleway.opposite|highway.residential' (first occurence for edge '4786965#0').
(pid=9782) Warning: Discarding unknown compound 'cycleway.opposite_track' in type 'cycleway.opposite_track|highway.residential' (first occurence for edge '4786972#0').
(pid=9782) Warning: Discarding unknown compound 'cycleway.opposite' in type 'cycleway.opposite|highway.living_street' (first occurence for edge '4795758').
(pid=9782) Warning: Discarding unknown compound 'cycleway.lane' in type 'cycleway.lane|highway.secondary' (first occurence for edge '100822066').
(pid=9782) Warning: Removed a road without junctions: -21549451
(pid=9782) Warning: Removed a road without junctions: 21549451
(pid=9782) Warning: The traffic light '30624291' does not control any links; it wi

(pid=9784) 2020-03-27 14:37:38,096	INFO dynamic_tf_policy.py:324 -- Initializing loss function with dummy input:
(pid=9784) 
(pid=9784) { 'action_prob': <tf.Tensor 'default_policy/action_prob:0' shape=(?,) dtype=float32>,
(pid=9784)   'actions': <tf.Tensor 'default_policy/actions:0' shape=(?, 2) dtype=float32>,
(pid=9784)   'advantages': <tf.Tensor 'default_policy/advantages:0' shape=(?,) dtype=float32>,
(pid=9784)   'behaviour_logits': <tf.Tensor 'default_policy/behaviour_logits:0' shape=(?, 4) dtype=float32>,
(pid=9784)   'dones': <tf.Tensor 'default_policy/dones:0' shape=(?,) dtype=bool>,
(pid=9784)   'new_obs': <tf.Tensor 'default_policy/new_obs:0' shape=(?, 26) dtype=float32>,
(pid=9784)   'obs': <tf.Tensor 'default_policy/observation:0' shape=(?, 26) dtype=float32>,
(pid=9784)   'prev_actions': <tf.Tensor 'default_policy/action:0' shape=(?, 2) dtype=float32>,
(pid=9784)   'prev_rewards': <tf.Tensor 'default_policy/prev_reward:0' shape=(?,) dtype=float32>,
(pid=9784)   'rewards': 

2020-03-27 14:37:43,624	ERROR import_thread.py:89 -- ImportThread: Error 111 connecting to 10.0.2.15:40655. Connection refused.
2020-03-27 14:37:43,635	ERROR worker.py:1716 -- listen_error_messages_raylet: Error 111 connecting to 10.0.2.15:40655. Connection refused.
2020-03-27 14:37:43,642	ERROR worker.py:1616 -- print_logs: Error 111 connecting to 10.0.2.15:40655. Connection refused.


KeyboardInterrupt: 